## Expenses

This notebook tries to visualize your annual expenses.

### How to use

 - `YEAR` is the year.
 - `EXPENSE_FILEPATH` is the path to a TOML file containing all your expenses
 - `FONT_FAMILY` for font type
 - `FONT_SIZE` for font size

### Toml format
Expenses should be listed as such:
```toml
[[expense]]
date = 2021-01-10T00:00:00-05:00
description = "Toilet Seat"
price = 28.23
source = "Home Depot"
categories = ["Home Appliance"]

[[expense]]
date = 2021-11-03T12:32:00-05:00
description = "Bubble Tea with friend"
price = 8.69
source = "Blossoming Juice"
categories = ["Bubble Tea"]
```


In [ ]:
## configurations

YEAR = 2021

EXPENSE_FILENAME_FORMAT = "TOML"

EXPENSE_DIRECTORY = f"{YEAR}/expenses"
EXPENSE_FILENAME = "expenses.toml"
EXPENSE_FILEPATH = f"{EXPENSE_DIRECTORY}/{EXPENSE_FILENAME}"

FONT_FAMILY = "DejaVu Sans"
FONT_SIZE = 22



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from enum import Enum
import toml

## classes

class Expense:
    def __init__(self, price=0.0, description="", category=set()):
        self.price = price
        self.description = description
        self.category = category

In [ ]:
## functions

def get_expenses(filename):
    L = [[] for i in range(12)]
    data = toml.load(filename)
    for d in data['expense']:
        L[d['date'].month - 1].append(d)
    return L


In [ ]:
# load

expenses = get_expenses(EXPENSE_FILEPATH)

## Monthly stats

In [ ]:
months = np.arange(1, 13)

monthly_expenses = [sum(e['price'] for e in m) for m in expenses]
monthly_aggregate_expenses = np.cumsum(monthly_expenses)

for i, m in enumerate(monthly_expenses):
    print("{:02d}\t$ {:.02f}".format(i + 1, m))
print("Total:\t$ {}".format(sum(monthly_expenses)))

font = {'family' : FONT_FAMILY,
        'weight' : 'bold',
        'size'   : FONT_SIZE}
plt.rc('font', **font)

fig = plt.figure(figsize=(24, 10))
ax = fig.add_axes([0, 0, 1, 1])

ax.set_title("{} Monthly Expenses".format(YEAR))
ax.set_ylabel("CAD $", fontsize=FONT_SIZE)
ax.set_xlabel("Month", fontsize=FONT_SIZE)
ax.set_xticks(months)

# plt.plot(months, monthly_expenses)
ax.fill_between(months, 0, monthly_aggregate_expenses, color="#A89B8C")
ax.fill_between(months, 0, monthly_expenses, color="#F0DFAD")

for (i, v) in enumerate(monthly_aggregate_expenses):
    ax.text(i + 0.7, v + 2,
            "{:.02f}".format(v),
            color='#1E2749',
            fontweight='normal',
            fontsize=FONT_SIZE)
    
for (i, v) in enumerate(monthly_expenses):
    ax.text(i + 0.7, v + 2,
            "{:.02f}".format(v),
            color='#1E2749',
            fontweight='normal',
            fontsize=FONT_SIZE)

## Category Stats

In [ ]:
import itertools

CATEGORY_QUANTITY_KEY = 0
CATEGORY_PRICE_KEY = 1

FONT_SIZE = 24

all_categories = set()
all_category_values = {}

for monthly_expenses in expenses:
    for expense in monthly_expenses:
        expense_categories = expense.get('categories', [])
        all_categories.update(expense_categories)
        for category in expense_categories:
            if category not in all_category_values:
                all_category_values[category] = {CATEGORY_QUANTITY_KEY: 0, CATEGORY_PRICE_KEY: 0.0}

            all_category_values[category][CATEGORY_QUANTITY_KEY] += 1
            all_category_values[category][CATEGORY_PRICE_KEY] += expense['price']

for (k, v) in all_category_values.items():
    print(k.ljust(20), '\t{}'.format(v[0]), "\t{:.02f}".format(v[1]))

monthly_expenses = [sum(e['price'] for e in m) for m in expenses]

print("Total:".ljust(20), '\t$\t{}'.format(sum(monthly_expenses)))

sorted_by_price = [(category, v[CATEGORY_PRICE_KEY]) for (category, v) in all_category_values.items()]
sorted_by_price.sort(key=lambda x: x[1])

prices = [price for (category, price) in sorted_by_price]
xlabels = [category for (category, price) in sorted_by_price]

fig = plt.figure(figsize=(40, 20))
ax = fig.add_axes([0, 0, 1, 1])

ax.set_title("{} Category Expenses".format(YEAR))
ax.set_ylabel("CAD $")
ax.set_xlabel("Categories")
ax.set_xticks(np.arange(len(xlabels)))
ax.set_xticklabels(xlabels, fontsize=FONT_SIZE)

xs = [i for i in range(0, len(all_categories))]

ax.bar(xs, prices, color="#A89B8C")